In [ ]:
!mkdir test

In [ ]:
!pip3 install symspellpy
!pip3 install tqdm
!pip3 install nltk
!sudo apt-get install rdfind

In [ ]:
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron1.tar.gz"
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron2.tar.gz"
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron3.tar.gz"
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron4.tar.gz"
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron5.tar.gz"
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron6.tar.gz"
!wget -q "http://nlp.cs.aueb.gr/software_and_datasets/lingspam_public.tar.gz"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20030228_spam.tar.bz2"
!wget -q "https://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
!tar -xzf enron1.tar.gz
!tar -xzf enron2.tar.gz
!tar -xzf enron3.tar.gz
!tar -xzf enron4.tar.gz
!tar -xzf enron5.tar.gz
!tar -xzf enron6.tar.gz
!tar -xzf lingspam_public.tar.gz

!tar -xjf 20021010_easy_ham.tar.bz2
!mv easy_ham 20021010_easy_ham

!tar -xjf 20021010_hard_ham.tar.bz2
!mv hard_ham 20021010_hard_ham

!tar -xjf 20021010_spam.tar.bz2
!mv spam 20021010_spam

!tar -xjf 20030228_easy_ham.tar.bz2
!mv easy_ham 20030228_easy_ham

!tar -xjf 20030228_easy_ham_2.tar.bz2
!mv easy_ham_2 20030228_easy_ham_2

!tar -xjf 20030228_hard_ham.tar.bz2
!mv hard_ham 20030228_hard_ham

!tar -xjf 20030228_spam.tar.bz2
!mv spam 20030228_spam

!tar -xjf 20030228_spam_2.tar.bz2
!mv spam_2 20030228_spam_2

In [ ]:
rm enron1.tar.gz enron2.tar.gz enron3.tar.gz enron4.tar.gz enron5.tar.gz enron6.tar.gz lingspam_public.tar.gz

In [ ]:
rm 20021010_easy_ham.tar.bz2 20021010_hard_ham.tar.bz2 20021010_spam.tar.bz2 20030228_easy_ham.tar.bz2 20030228_easy_ham_2.tar.bz2 20030228_hard_ham.tar.bz2 20030228_spam.tar.bz2 20030228_spam_2.tar.bz2

In [ ]:
!rdfind -deleteduplicates true .

In [ ]:
!find . -name "cmds" -delete
!find . -name "0000.7b1b73cf36cf9dbc3d64e3f2ee2b91f1" -delete

In [ ]:
# import libraries
import numpy as np
import csv
import os
import pandas as pd
import string
import matplotlib.pyplot as plt

# Sort BoW in descending order
import heapq

# Process raw html files
import re
from html import unescape
import email
import email.policy
from email.parser import BytesParser, Parser
from email.policy import default

# Give time count for processing certain tasks
import tqdm

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')

from symspellpy import SymSpell, Verbosity

# Give root folder where CS20S020_assignment3_dataset.zip is extracted.
# Set this to '/content/' if running on colab
PATH = '/content/'
global PATH

In [ ]:
np.random.seed(11)

In [ ]:
# Import raw dataset
def import_datasets():

  # All unprocessed enron and lingspam emails are directly converted into a pandas dataframe.
    
  rows = []
  spam_count = 0
  ham_count = 0

  enron_files = ['enron1', 'enron2', 'enron3', 'enron4', 'enron5', 'enron6']
  path = PATH

  for enron_file in enron_files:
    for directories, subdirs, files in os.walk(path + enron_file):
      for filename in files:
        with open(os.path.join(directories, filename), encoding="utf-8", errors="ignore") as f:
          data = f.read()
          if (os.path.split(directories)[1]  == 'ham'):
            rows.append({'email': data, 'class': 'ham'})
            ham_count += 1
    
          if (os.path.split(directories)[1]  == 'spam'):
            rows.append({'email': data, 'class': 'spam'})
            spam_count += 1

  print("Distribution of Enron emails")
  print("#Ham emails: ", ham_count, "#Spam emails: ", spam_count)
  
  spam_count = 0
  ham_count = 0
  
  path = PATH + '/lingspam_public/bare/'
  lingspam_folders = ['part1', 'part2', 'part3', 'part4', 'part5', 'part6',
                    'part7', 'part8', 'part9', 'part10']

  for folder in lingspam_folders:
    for directories, subdirs, text_files in os.walk(path + folder):
      for filename in text_files:
        with open(os.path.join(directories, filename), encoding="utf-8", errors="ignore") as f:
          data = f.read()
          if 'spmsg' in filename:
            rows.append({'email': data, 'class': 'spam'})
            spam_count += 1
          else:
            rows.append({'email': data, 'class': 'ham'})
            ham_count += 1

  print("Distribution of Lingspam emails")
  print("#Ham emails: ", ham_count, "#Spam emails: ", spam_count)

  spamassasino = ['20021010_easy_ham/', '20021010_hard_ham/', '20021010_spam/', '20030228_easy_ham/', '20030228_easy_ham_2/', 
              '20030228_hard_ham/', '20030228_spam/', '20030228_spam_2/', '20050311_spam_2/']
  path = PATH
  index = []

  spam_count = 0
  ham_count = 0

  for folder in spamassasino:
    folder_path = path + folder
    for directories, subdirs, files in os.walk(folder_path):
      for filename in files:
        with open(os.path.join(folder_path, filename), "rb") as f:
          try:
            #data = BytesParser(policy=default).parse(f)
            data = email.message_from_binary_file(f)
            
            text = ""
            emtext = ""
            
            if data.is_multipart() or 'multipart' in data.get_content_type():
              html = ""

              for part in data.get_payload():
                  if part.get_content_type() == 'text/plain' or part.get_content_type() == 'text/html':
                    
                    if part.get_content_charset() is None:
                      charset = chardet.detect(str(part))['encoding']
                    else:
                      charset = part.get_content_charset()

                    if part.get_content_type() == 'text/plain':
                      text += str(part.get_payload(decode=True), str(charset), 'ignore')
                    if part.get_content_type() == 'text/html':
                      text = htmlTOtext(part.get_payload())
                      html += text
              
              data = ""
              if html != "" and text != "":
                emtext += html.strip() + '\n' + text.strip()
                data = emtext #.encode('utf-8', 'ignore')

              elif html == "" and text == "":
                data = None

              else:
                if html == "":
                  emtext = text.strip()
                else:
                  emtext = html.strip()
                data = emtext #.encode('utf-8', 'ignore')

            else:
              if data.get_content_type() == 'text/html':
                text = htmlTOtext(data.get_payload())
                #text = text.encode('utf-8', errors='ignore')
              elif data.get_content_type() == 'text/plain':
                if data.get_charsets() == [None]:
                  text = str(data.get_payload(decode=True), 'utf-8', 'ignore') #.encode('utf-8','ignore')
                else:
                  text = str(data.get_payload(decode=True), data.get_content_charset(), 'ignore') #.encode('utf-8','ignore')
              data = text.strip()

            if data is not None:
              if 'easy_ham' in folder or 'easy_ham_2' in folder or 'hard_ham' in folder:
                rows.append({'email': data, 'class': 'ham'})
                ham_count += 1

              if 'spam' in folder or 'spam_2' in folder:
                rows.append({'email': data, 'class': 'spam'})
                spam_count += 1
              index.append(folder_path+filename)
          except:
            continue

  print("Distribution of Spamassasin emails")
  print("#Ham emails: ", ham_count, "#Spam emails: ", spam_count)
  return rows, index

In [ ]:
def clean_email(emails_array):
    """ Remove all punctuation, urls, numbers, and newlines.
    Convert to lower case."""

    i = 0
    for email in emails_array:
      if type(email) == 'bytes':
        email = str(email, encoding='ascii', errors='ignore')
      else:
        email = str(email)
      email = email.lower()
      email = re.sub(r'\\r', ' ', email)
      email = re.sub(r'\\n', ' ', email)
      email = re.sub(r'http\S+', 'httplink', email)
      email = re.sub(r'[^\s]+@[^\s]+', 'emailaddr', email)
      email = re.sub(r'[^\s]+\.(gif|png|jpg|jpeg)$', 'imgext', email)
      email = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', 'ipaddr', email)
      email = re.sub(r'[^\S]', ' ', email)
      email = re.sub(r' +', ' ', email)
      #The '$' sign gets replaced with 'dollar'
      email = re.sub('[$]+', 'dollar', email)

      email = email.translate(str.maketrans("", "", string.punctuation))
      
      email = re.sub("\d+", ' ', email)
      email = email.replace('\n', ' ')

      email = email.strip()
      emails_array[i] = email
      i += 1

    return emails_array

In [ ]:
def count_all_words(emails_array):
  # Word frequency
  word2count = {}
  for email in emails_array:
    words = email.split()
    for word in words:
      if word not in word2count.keys():
        word2count[word] = 1
      else:
        word2count[word] += 1

  return word2count

In [ ]:
def remove_stopwords(emails_array):
  # open file and read the content in a list. 
  # Remove stopwords such as 'the','you', 'an' etc.
  # Also, removes words of length < 2 or length > 15.
  
  with open(PATH + 'stopwords.txt', 'r') as f:
    sw = []
    for line in f:
      sw.append(line[:-1])

  i = 0
  for email in emails_array:
    words = email.split()
    email = ""
    for word in words:
      if word not in sw and len(word) > 2 and len(word) < 15:
        email += word + " "
    if email != "":
      email = re.sub(r'[^\S]', ' ', email)
      email = re.sub(r' +', ' ', email)
      email = email.strip()
      emails_array[i] = email
    i += 1

  return emails_array  

In [ ]:
def lemmatize_stem(emails_array, train=True):
  # Words are stemmized and lemmatized
  i = 0
  for email in emails_array:

    # Create the stemmer, lemmatizer
    wnl = WordNetLemmatizer()
    s_stemmer = SnowballStemmer(language='english')

    # Split text into words.
    words = email.split()
    email = ""
    for word in words:
        x = s_stemmer.stem(word)
        x = wnl.lemmatize(x)
        email += x + " "
    emails_array[i] = email
    i += 1

  emails_array = remove_stopwords(emails_array)

  # Remove low frequency words only for train emails
  if train:
    emails_array = remove_low_freq_words(emails_array)    

  return emails_array

In [ ]:
def remove_low_freq_words(emails_array):

  # Get frequency for all words in our dataset
  word2count = count_all_words(emails_array)

  # Remove all words with freq <= 3
  i = 0
  for email in emails_array:
    words = email.split()
    email = ""
    for word in words:
      if word2count[word] >= 3:
        email += word + " "
    email = re.sub(r'[^\S]', ' ', email)
    email = re.sub(r' +', ' ', email)
    email = email.strip()
    
    emails_array[i] = email
    i += 1
    
  return emails_array

In [ ]:
def spell_check(emails_array, labels):
  
  emails_array = remove_stopwords(emails_array)

  # Removing low frequency words
  emails_array = remove_low_freq_words(emails_array)
  
  nspam_emails = emails_array[np.where(labels == 0)]
  word2count = count_all_words(nspam_emails) # count_all_words(emails_array)
  
  dist = 3
  dictionary_path = PATH + "/frequency_dictionary_en_82_765.txt"
  sym_spell = SymSpell(max_dictionary_edit_distance=dist, prefix_length=dist+1, count_threshold=1)
  sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

  # finding correctly spelled words
  spell_correct_all_words = {}
  for word in tqdm.tqdm(word2count.keys()):
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=dist)

    if len(suggestions) > 0:
      suggestion = str(suggestions[0]).split(',')[0]
      if word != suggestion:
        spell_correct_all_words[word] = suggestion
    else:
      spell_correct_all_words[word] = '0'

  # Removing no spell found words and correcting words
  i = 0
  for email in emails_array:
    words = email.split()
    for word in words:
      if (word in spell_correct_all_words.keys()) and spell_correct_all_words[word] != '0':
        email = email.replace(word, spell_correct_all_words[word])
      if (word in spell_correct_all_words.keys()) and spell_correct_all_words[word] == '0':
        email = email.replace(word, "")
    
    email = re.sub(r'[^\S]', ' ', email)
    email = re.sub(r' +', ' ', email)
    email = email.strip()
    
    emails_array[i] = email
    i += 1
  
  emails_array = remove_stopwords(emails_array)

  return emails_array, spell_correct_all_words

In [ ]:
# Creating the Bag of Words model
def create_bag_of_words(emails_array, labels, max_words):
  spam_words = {}
  nspam_words = {}

  spam_emails = emails_array[np.where(labels == 1)]
  nspam_emails = emails_array[np.where(labels == 0)]

  spam_words = count_all_words(spam_emails)
  nspam_words = count_all_words(nspam_emails)

  # If there are common keys in both spam and non-spam list, then we assign the key
  # to that list which has a higher frequency
  common_words = set(spam_words.keys()).intersection(set(nspam_words.keys()))
  if len(common_words) > 0:
    for key in common_words:
      if spam_words[key] > nspam_words[key]:
        nspam_words.pop(key)
      else:
        spam_words.pop(key)
      
  top_k_spam = heapq.nlargest(int(max_words/2), spam_words, key=spam_words.get)
  top_k_nspam = heapq.nlargest(int(max_words/2), nspam_words, key=nspam_words.get)
  freq_words = top_k_spam + top_k_nspam
  
  return freq_words, spam_words, nspam_words

In [ ]:
def tokenize(emails_array, freq_words, tf=False):
  # Tokenize words based on BoW. This method gives the term-frequency in
  # binarized form if tf is False, or raw term-frequency if tf is True

  X = []
  for email in emails_array:
    vector = []
    email_tokenization = email.split()
    for word in freq_words:
      if word in email_tokenization:
        if tf:
          c = 0
          for w in freq_words:
            if word in w:
              c += 1
          vector.append(c)
        else:
          vector.append(1)
      else:
        vector.append(0)
    X.append(vector)
  X = np.asarray(X)

  return X

In [ ]:
def htmlTOtext(html):
    # Converts raw html email into plain text

    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text) 

In [ ]:
def remove_empty_emails(emails_array):
  # Empty emails are removed from training set.  

  if emails_array.ndim > 1:
    ems = emails_array[:, 0]
  elif emails_array.ndim == 1:
    ems = emails_array
  
  i = 0
  idxs = []
  for em in ems:
    if len(em) == 0:
      idxs.append(i)
    i += 1

  if len(idxs) > 0:
    emails_array = np.delete(emails_array, np.array(idxs), axis=0)
    
  return emails_array, idxs

In [ ]:
def remove_empty_tokenized_emails(token_emails, emails_array):
  # Removes zero-vector tokenized emails during training

  idxs = np.where(np.sum(token_emails, axis=1) == 0)[0]
  
  if len(idxs) > 0:
    token_emails = np.delete(token_emails, idxs, axis=0)
    emails_array = np.delete(emails_array, idxs, axis=0)

  return token_emails, emails_array, idxs

In [ ]:
# Extract test emails for evaluation.
# 'Upload emails in test folder. Drag and drop'
def get_test_emails(path):
  folder_path = path

  rows = []

  for f in os.listdir(folder_path):
    with open(folder_path + f, encoding="utf-8", errors="ignore") as f:
      data = f.read()
      rows.append({'email': data})

  return rows

In [ ]:
class BernNB(object):
    def __init__(self, alpha=1.0, binarize=0.5):
        self.alpha = alpha
        self.binarize = binarize

    def fit(self, X, y):
        X = self.binarize_X(X)

        num_samples = X.shape[0]

        # Group by labels
        group_by_label = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]

        # Label prior
        self.class_log_prior_ = [np.log(len(i) / num_samples) for i in group_by_label]

        # Each word's count
        word_count = np.array([np.array(i).sum(axis=0) for i in group_by_label]) + self.alpha

        smoothing = 2 * self.alpha

        # number of documents in each class + smoothing
        num_docs = np.array([len(i) + smoothing for i in group_by_label])
        
        # probability of each word
        self.feature_prob_ = word_count / num_docs[np.newaxis].T
        return self

    def predict_log_probs(self, X):
        X = self.binarize_X(X)
        return [(np.log(self.feature_prob_) * x + \
                 np.log(1 - self.feature_prob_) * np.abs(x - 1)
                ).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        X = self.binarize_X(X)
        return np.argmax(self.predict_log_probs(X), axis=1)
    
    def binarize_X(self, X):
      return np.where(X > self.binarize, 1, 0) if self.binarize != None else X

    def score(self, pred, true):
      return sum(pred == true) / len(true)

In [ ]:
# Download and concatenate dataset into a dataframe
print("Importing datasets....")
rows, index = import_datasets()

emails_dataset = pd.DataFrame(rows)

# Map 'spam' to 1 and 'ham' to 0.
emails_dataset['class'] = emails_dataset['class'].map({'spam':1, 'ham':0})

In [ ]:
emails = emails_dataset.to_numpy()
train_emails = emails

# Clean emails
print("Cleaning training emails....")
train_emails[:, 0] = clean_email(train_emails[:, 0])

In [ ]:
print("Performing spellcheck....")
# Perform spell check on non-spam words
train_emails[:, 0], spell_correct_all_words = spell_check(train_emails[:, 0], train_emails[:, 1])

# Remove stopwords
train_emails[:, 0] = remove_stopwords(train_emails[:, 0])

print("Stemming and Lemmatizing....")
# Stemmatize and Lemmatize all words in all emails
train_emails[:, 0] = lemmatize_stem(train_emails[:, 0])

In [ ]:
# Remove empty emails
train_emails, _ = remove_empty_emails(train_emails)

# Shuffle training set randomly
np.random.shuffle(train_emails)

In [ ]:
# Get BoW
print("Creating BoW....")
BoW, spam_words, nspam_words = create_bag_of_words(train_emails[:, 0], train_emails[:, 1], 5000)

In [ ]:
# Vectorize words
tokenized_emails = tokenize(train_emails[:, 0], BoW)

# Remove zero-vectors emails from training set
tokenized_emails, train_emails, _ = remove_empty_tokenized_emails(tokenized_emails, train_emails)

# Laplace smoothing
tokenized_emails = np.append(tokenized_emails, [np.ones(len(BoW))], 0)
tokenized_emails = np.append(tokenized_emails, [np.ones(len(BoW))], 0)
X_train = tokenized_emails

# Define the target labels
y_train = train_emails[:, 1]
y_train = np.append(y_train, 0)
y_train = np.append(y_train, 1)
y_train = y_train.astype('int')

In [ ]:
path = PATH + '/test/'
# Read test emails
rows = get_test_emails(path)
test_df = pd.DataFrame(rows)

test_ems = test_df['email'].values
# Clean test emails and remove stopwords
test_ems = clean_email(test_ems)
test_ems = remove_stopwords(test_ems)
_, idxs1 = remove_empty_emails(test_ems)

# Vectorize words
tokenized_tests = tokenize(test_ems, BoW)

# Remove zero-vectors emails from test set
_, _, idxs2 = remove_empty_tokenized_emails(tokenized_tests, test_ems)

X_test = tokenized_tests

In [ ]:
bnb = BernNB(alpha=1, binarize=0.5).fit(X_train, y_train)
test_predictions = bnb.predict(X_test)

# Setting zero-vector emails by default to ham or label '0'
if len(idxs1) > 0:
  test_predictions[idxs1] = 0
if len(idxs2) > 0:
  test_predictions[idxs2] = 0

# Output returned to test folder
with open(path+'output.txt', 'w', newline='') as file_output:
  for item in test_predictions:
    file_output.write("%s\n" % item)